<a href="https://colab.research.google.com/github/Swayamprakashpatel/HPLC/blob/main/HPLC_RT_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HPLC Rt Predictor**

> *Ramanbhai Patel College of Pharmacy*

> Dr. Swayamprakash Patel (swayamprakashpatel.ph@charusat.ac.in)

> Dr. Mehul Patel (mehulpatel.ph@charusat.ac.in)

**To predict RT**

In [ ]:

!pip install pubchempy
import tensorflow as tf
import pandas as pd
import numpy as np
import io
from tensorflow import keras
import pubchempy as pcp
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import time as tm

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1TYa0HCjzRk9SxBe8hslYVsr9Hahge6Z4',
                                   dest_path='/content/sample_data/HPLC_NUM.hdf5',
                                   unzip=False)
#Load Model
model = load_model('/content/sample_data/HPLC_NUM.hdf5', compile= True)




Drug_1_CID =  24872560#@param {type:"number"}
Drug_2_CID =  24872560#@param {type:"number"}
Methanol =  0#@param {type:"number"}
Methanol = Methanol/100
Acetonitrile =  52#@param {type:"number"}
Acetonitrile = Acetonitrile/100
Water =  0#@param {type:"number"}
Water = Water/100
Buffer =  48#@param {type:"number"}
Buffer = Buffer/100
pH =  2.5#@param {type:"number"}
pH = pH/14


Drug_1_CID = pcp.Compound.from_cid(Drug_1_CID)
Drug_2_CID = pcp.Compound.from_cid(Drug_2_CID)

FPD_1 = Drug_1_CID.cactvs_fingerprint
FPD_2 = Drug_2_CID.cactvs_fingerprint

List1 = [Methanol, Acetonitrile, Water, Buffer, pH]

List2 = list(FPD_1)
List3 = list(FPD_2)
List = List1+List2+List3
t = pd.DataFrame(np.array(List).reshape(-1,len(List)))

dataset1 = t.values
t.dtype = int
X_Predict = (dataset1[:,0:1767].astype(float))


Y_prediction = model.predict(X_Predict)
Y_prediction = Y_prediction
#Y_prediction = pd.DataFrame(Y_prediction)
print(Y_prediction)


**To Predict System**

In [ ]:
!pip install tabulate
!pip install pubchempy
import pubchempy as pcp
import pandas as pd
import numpy as np
from keras.models import load_model
from google_drive_downloader import GoogleDriveDownloader as gdd
from tabulate import tabulate

# Download and load the model
gdd.download_file_from_google_drive(file_id='1VCo8GVaTW6CAVaezBpkku3UzWebpRBzH',
                                    dest_path='/content/sample_data/HPLC_System.hdf5',
                                    unzip=False)
model = load_model('/content/sample_data/HPLC_System.hdf5', compile=True)

# Define a function to convert predictions to meaningful values
def process_predictions(predictions):
    predictions[:, 0:4] *= 100  # Multiply first four columns by 100
    predictions[:, 4] *= 14      # Multiply pH column by 14
    return predictions

# Define compound CIDs
Drug_1_CID =  24872560#@param {type:"number"}
Drug_2_CID =  24872560#@param {type:"number"}

# Fetch compounds using pubchempy
Drug_1_CID = pcp.Compound.from_cid(Drug_1_CID)
Drug_2_CID = pcp.Compound.from_cid(Drug_2_CID)

# Extract fingerprints
FPD_1 = Drug_1_CID.cactvs_fingerprint
FPD_2 = Drug_2_CID.cactvs_fingerprint

# Concatenate the fingerprints
List2 = list(FPD_1)
List3 = list(FPD_2)
List = List2 + List3

# Create a DataFrame for predictions
t = pd.DataFrame(np.array(List).reshape(-1, len(List)))

# Convert DataFrame to int type
t = t.astype(int)

# Extract the necessary columns for prediction
X_Predict = t.iloc[:, 0:1762].astype(float).values

# Make predictions
Y_prediction = model.predict(X_Predict)

# Process predictions to meaningful values
processed_predictions = process_predictions(Y_prediction)

# Define column labels
columns = ['Methanol (%)', 'Acetonitrile (%)', 'Water (%)', 'Buffer (%)', 'pH']

# Create a DataFrame with labels and processed predictions
result_df = pd.DataFrame(data=processed_predictions, columns=columns)

# Generate a formatted table using tabulate
table = tabulate(result_df, headers='keys', tablefmt='grid', showindex=False)

# Print the formatted table
print(table)
